In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 4
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.05 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_5S
            epoch = 90
            lr = 0.01
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.01
        else:
            trainData = data_1S
            epoch = 90
            lr = 0.01
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 2.0345940589904785 Train_acc: 23.437500%
Train Epoch: 2/90 Train_Loss: 1.4362635612487793 Train_acc: 43.437500%
Train Epoch: 3/90 Train_Loss: 1.103327989578247 Train_acc: 66.562500%
Train Epoch: 4/90 Train_Loss: 0.6689718961715698 Train_acc: 67.500000%
Train Epoch: 5/90 Train_Loss: 0.5926269292831421 Train_acc: 78.125000%
Train Epoch: 6/90 Train_Loss: 0.3411543667316437 Train_acc: 86.250000%
Train Epoch: 7/90 Train_Loss: 0.4159713685512543 Train_acc: 90.625000%
Train Epoch: 8/90 Train_Loss: 0.36276018619537354 Train_acc: 92.812500%
Train Epoch: 9/90 Train_Loss: 0.1454080492258072 Train_acc: 94.375000%
Train Epoch: 10/90 Train_Loss: 0.18329429626464844 Train_acc: 97.500000%
Train Epoch: 11/90 Train_Loss: 0

Train Epoch: 13/90 Train_Loss: 0.013419046066701412 Train_acc: 97.291672%
Train Epoch: 14/90 Train_Loss: 0.04411069676280022 Train_acc: 98.020836%
Train Epoch: 15/90 Train_Loss: 0.12077342718839645 Train_acc: 98.020836%
Train Epoch: 16/90 Train_Loss: 0.010428017005324364 Train_acc: 98.854172%
Train Epoch: 17/90 Train_Loss: 0.09767258912324905 Train_acc: 98.645836%
Train Epoch: 18/90 Train_Loss: 0.01744907908141613 Train_acc: 98.229172%
Train Epoch: 19/90 Train_Loss: 0.012168260291218758 Train_acc: 98.020836%
Train Epoch: 20/90 Train_Loss: 0.04262564703822136 Train_acc: 98.125008%
Train Epoch: 21/90 Train_Loss: 0.04464058578014374 Train_acc: 98.750008%
Train Epoch: 22/90 Train_Loss: 0.09135905653238297 Train_acc: 98.854172%
Train Epoch: 23/90 Train_Loss: 0.06111031398177147 Train_acc: 98.854172%
Train Epoch: 24/90 Train_Loss: 0.010587524622678757 Train_acc: 98.750008%
Train Epoch: 25/90 Train_Loss: 0.038470398634672165 Train_acc: 98.645836%
Train Epoch: 26/90 Train_Loss: 0.0082937739789

Train Epoch: 30/90 Train_Loss: 0.10075268894433975 Train_acc: 95.125000%
Train Epoch: 31/90 Train_Loss: 0.04412977397441864 Train_acc: 95.000000%
Train Epoch: 32/90 Train_Loss: 0.08978766947984695 Train_acc: 95.437500%
Train Epoch: 33/90 Train_Loss: 0.08454303443431854 Train_acc: 95.375000%
Train Epoch: 34/90 Train_Loss: 0.14721904695034027 Train_acc: 95.375000%
Train Epoch: 35/90 Train_Loss: 0.12552884221076965 Train_acc: 94.875000%
Train Epoch: 36/90 Train_Loss: 0.10519886016845703 Train_acc: 95.375000%
Train Epoch: 37/90 Train_Loss: 0.044968605041503906 Train_acc: 95.312500%
Train Epoch: 38/90 Train_Loss: 0.19710837304592133 Train_acc: 95.437500%
Train Epoch: 39/90 Train_Loss: 0.13586506247520447 Train_acc: 95.375000%
Train Epoch: 40/90 Train_Loss: 0.24018462002277374 Train_acc: 95.500000%
Train Epoch: 41/90 Train_Loss: 0.08371621370315552 Train_acc: 95.437500%
Train Epoch: 42/90 Train_Loss: 0.04465373605489731 Train_acc: 95.187500%
Train Epoch: 43/90 Train_Loss: 0.11091042309999466

Train Epoch: 47/90 Train_Loss: 0.002027949085459113 Train_acc: 100.000000%
Train Epoch: 48/90 Train_Loss: 0.002015126869082451 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.0016879666363820434 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.0022563033271580935 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Loss: 0.0016287127509713173 Train_acc: 100.000000%
Train Epoch: 52/90 Train_Loss: 0.0018112794496119022 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.0021205826196819544 Train_acc: 100.000000%
Train Epoch: 54/90 Train_Loss: 0.004810412414371967 Train_acc: 100.000000%
Train Epoch: 55/90 Train_Loss: 0.0012908369535580277 Train_acc: 100.000000%
Train Epoch: 56/90 Train_Loss: 0.0020939053501933813 Train_acc: 100.000000%
Train Epoch: 57/90 Train_Loss: 0.005817951634526253 Train_acc: 100.000000%
Train Epoch: 58/90 Train_Loss: 0.003079668851569295 Train_acc: 100.000000%
Train Epoch: 59/90 Train_Loss: 0.0020846331026405096 Train_acc: 100.000000%
Train Epoch: 60/9

Train Epoch: 63/90 Train_Loss: 0.027304155752062798 Train_acc: 98.854172%
Train Epoch: 64/90 Train_Loss: 0.00047476807958446443 Train_acc: 98.854172%
Train Epoch: 65/90 Train_Loss: 0.03824267163872719 Train_acc: 98.854172%
Train Epoch: 66/90 Train_Loss: 0.02946512959897518 Train_acc: 98.854172%
Train Epoch: 67/90 Train_Loss: 0.007794506382197142 Train_acc: 98.854172%
Train Epoch: 68/90 Train_Loss: 0.01969030313193798 Train_acc: 98.854172%
Train Epoch: 69/90 Train_Loss: 0.0015982226468622684 Train_acc: 98.854172%
Train Epoch: 70/90 Train_Loss: 0.04305450990796089 Train_acc: 98.854172%
Train Epoch: 71/90 Train_Loss: 0.0229661725461483 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.02289886586368084 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.021362537518143654 Train_acc: 98.854172%
Train Epoch: 74/90 Train_Loss: 0.0005906690494157374 Train_acc: 98.854172%
Train Epoch: 75/90 Train_Loss: 0.022735364735126495 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.04981712

...

Train Epoch: 35/90 Train_Loss: 0.001772908610291779 Train_acc: 100.000000%
Train Epoch: 36/90 Train_Loss: 0.007129174191504717 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.002167920581996441 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.002427713479846716 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.0018515132833272219 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.002008449286222458 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 0.0049641248770058155 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 0.003349913749843836 Train_acc: 100.000000%
Train Epoch: 43/90 Train_Loss: 0.0024182370398193598 Train_acc: 100.000000%
Train Epoch: 44/90 Train_Loss: 0.002751697786152363 Train_acc: 100.000000%
Train Epoch: 45/90 Train_Loss: 0.002330446382984519 Train_acc: 100.000000%
Train Epoch: 46/90 Train_Loss: 0.003829812863841653 Train_acc: 100.000000%
Train Epoch: 47/90 Train_Loss: 0.0022020931355655193 Train_acc: 100.000000%
Train Epoch: 48/90 Tr

Train Epoch: 50/90 Train_Loss: 0.0583110973238945 Train_acc: 98.645836%
Train Epoch: 51/90 Train_Loss: 0.03487395495176315 Train_acc: 98.645836%
Train Epoch: 52/90 Train_Loss: 0.005272039212286472 Train_acc: 98.645836%
Train Epoch: 53/90 Train_Loss: 0.046581167727708817 Train_acc: 98.645836%
Train Epoch: 54/90 Train_Loss: 0.02189866453409195 Train_acc: 98.645836%
Train Epoch: 55/90 Train_Loss: 0.08760195970535278 Train_acc: 98.645836%
Train Epoch: 56/90 Train_Loss: 0.05600813776254654 Train_acc: 98.645836%
Train Epoch: 57/90 Train_Loss: 0.0010391870746389031 Train_acc: 98.645836%
Train Epoch: 58/90 Train_Loss: 0.036332350224256516 Train_acc: 98.645836%
Train Epoch: 59/90 Train_Loss: 0.021921858191490173 Train_acc: 98.645836%
Train Epoch: 60/90 Train_Loss: 0.023308653384447098 Train_acc: 98.645836%
Train Epoch: 61/90 Train_Loss: 0.010787446983158588 Train_acc: 98.645836%
Train Epoch: 62/90 Train_Loss: 0.019782837480306625 Train_acc: 98.645836%
Train Epoch: 63/90 Train_Loss: 0.0213887561

Train Epoch: 67/90 Train_Loss: 0.08391318470239639 Train_acc: 95.000000%
Train Epoch: 68/90 Train_Loss: 0.05984114855527878 Train_acc: 95.000000%
Train Epoch: 69/90 Train_Loss: 0.0862494707107544 Train_acc: 95.000000%
Train Epoch: 70/90 Train_Loss: 0.04383261874318123 Train_acc: 95.000000%
Train Epoch: 71/90 Train_Loss: 0.11047745496034622 Train_acc: 95.062500%
Train Epoch: 72/90 Train_Loss: 0.07634598761796951 Train_acc: 95.125000%
Train Epoch: 73/90 Train_Loss: 0.06160308048129082 Train_acc: 95.000000%
Train Epoch: 74/90 Train_Loss: 0.050571028143167496 Train_acc: 94.875000%
Train Epoch: 75/90 Train_Loss: 0.13986431062221527 Train_acc: 95.000000%
Train Epoch: 76/90 Train_Loss: 0.1185733750462532 Train_acc: 94.875000%
Train Epoch: 77/90 Train_Loss: 0.05931388586759567 Train_acc: 95.000000%
Train Epoch: 78/90 Train_Loss: 0.05007702857255936 Train_acc: 95.000000%
Train Epoch: 79/90 Train_Loss: 0.09665991365909576 Train_acc: 95.000000%
Train Epoch: 80/90 Train_Loss: 0.12864449620246887 T

Train Epoch: 83/90 Train_Loss: 0.002267865464091301 Train_acc: 100.000000%
Train Epoch: 84/90 Train_Loss: 0.010960429906845093 Train_acc: 100.000000%
Train Epoch: 85/90 Train_Loss: 0.008044657297432423 Train_acc: 100.000000%
Train Epoch: 86/90 Train_Loss: 0.0015997583977878094 Train_acc: 100.000000%
Train Epoch: 87/90 Train_Loss: 0.002001773798838258 Train_acc: 100.000000%
Train Epoch: 88/90 Train_Loss: 0.0043966942466795444 Train_acc: 100.000000%
Train Epoch: 89/90 Train_Loss: 0.0015973938861861825 Train_acc: 100.000000%
Train Epoch: 90/90 Train_Loss: 0.0035749641247093678 Train_acc: 100.000000%
Test accuracy of the model: 45.250000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 85.000000%
Training Time: 30.64 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Ep

Train Epoch: 5/90 Train_Loss: 0.17762288451194763 Train_acc: 90.812500%
Train Epoch: 6/90 Train_Loss: 0.3470456004142761 Train_acc: 91.250000%
Train Epoch: 7/90 Train_Loss: 0.1923581212759018 Train_acc: 91.812500%
Train Epoch: 8/90 Train_Loss: 0.15322807431221008 Train_acc: 93.062500%
Train Epoch: 9/90 Train_Loss: 0.1422778069972992 Train_acc: 92.937500%
Train Epoch: 10/90 Train_Loss: 0.1281639039516449 Train_acc: 91.562500%
Train Epoch: 11/90 Train_Loss: 0.12008506804704666 Train_acc: 93.500000%
Train Epoch: 12/90 Train_Loss: 0.11388234049081802 Train_acc: 93.562500%
Train Epoch: 13/90 Train_Loss: 0.0881594642996788 Train_acc: 93.812500%
Train Epoch: 14/90 Train_Loss: 0.17350511252880096 Train_acc: 93.437500%
Train Epoch: 15/90 Train_Loss: 0.1430709958076477 Train_acc: 93.437500%
Train Epoch: 16/90 Train_Loss: 0.17280477285385132 Train_acc: 94.000000%
Train Epoch: 17/90 Train_Loss: 0.16802723705768585 Train_acc: 93.500000%
Train Epoch: 18/90 Train_Loss: 0.20036299526691437 Train_acc: 

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()